In [2]:
import numpy as np
import pandas as pd
import orangecontrib.associate.fpgrowth as oaf
import datetime
import argparse
#parser = argparse.ArgumentParser()
#parser.add_argument('--train_dir', type=str, default='./trainDataHJ.txt')
#parser.add_argument('--rules_dir', type=str, default='./rulesDataHJ.txt')
#parser.add_argument('--support', type=float, default=0.2)
#parser.add_argument('--confidence',type=float,default=0.2)
#args = parser.parse_args()

In [3]:
y=np.random.randint(1,6,[1000,])
area=np.random.uniform(1,10,[1000,])
length=np.random.uniform(1,10,[1000,])
metal_thi=np.random.uniform(0,2,[1000,])
metal_cat=np.random.randint(1,4,[1000,])
mach_cat=np.random.randint(1,4,[1000,])
meth_cat=np.random.randint(1,4,[1000,])


In [4]:
df=pd.DataFrame({
    "area":area,
    "length":length,
    "metal_thi":metal_thi,
    "metal_cat":metal_cat,
    "mach_cat":mach_cat,
    "meth_cat":meth_cat,
    "quality":y
})

In [7]:
#df.to_csv('trainDataHJ.txt',index=False)

In [8]:
def data_pre(path):
    df=pd.read_csv(path)
    #iscretization
    bins=[0,2,4,6,8,10]
    df['area']=pd.cut(df['area'],bins,right=False,labels=['1','2','3','4','5'])
    bins=[0,2,4,6,8,10]
    df['length']=pd.cut(df['length'],bins,right=False,labels=['1','2','3','4','5'])
    bins=[0,0.5,1,1.5,2]
    df['metal_thi']=pd.cut(df['metal_thi'],bins,right=False,labels=['1','2','3','4'])
    #type trans
    df['metal_cat']=df['metal_cat'].apply(str)
    df['mach_cat']=df['mach_cat'].apply(str)
    df['meth_cat']=df['meth_cat'].apply(str)
    df['quality']=df['quality'].apply(str)
    #one_hot
    df=pd.get_dummies(df)
    strDecode=df.columns.values.tolist()
    data=df.values
    data.dtype=bool
    return data,strDecode

In [9]:
def model(data,support=0.05,confidence=0.2):
    fre_ite=dict(oaf.frequent_itemsets(data,support))  #这里设置置信度
    rules = oaf.association_rules(fre_ite,confidence)
    result=list(rules)
    return result

In [15]:
def dealResult(rules,strDecode):
    returnRules = []
    for i in rules:
        j= list(i[1])
        if len(j)==1 and strDecode[j[0]].startswith('quality') :
            temStr = [];
            temp=''
            for k in i[0]:
                temp = temp+strDecode[k]+'&'
            temp = temp[:-1]
            temp = temp + ' ==> '
            temp = temp +strDecode[j[0]] 
            temStr.append(temp)
            temStr.append(i[2])
            temStr.append(i[3])
            returnRules.append(temStr)
    returnRules=list(sorted(returnRules,key=lambda x:(-x[2],-x[1])))
    return returnRules
    


In [16]:
support=0.05
confidence=0.2
train_dir='trainDataHJ.txt'
rules_dir='rulesDataHJ.txt'
data,strDecode=data_pre(train_dir)
result=model(data,support,confidence)
printRules= dealResult(result,strDecode)
file=open(rules_dir,'w')
print('Rules'+'\t'+'support'+'\t'+'confidence'+'\n')
file.write('Rules'+'\t'+'support'+'\t'+'confidence'+'\n')
for line in printRules:
    s='\t'.join([str(i) for i in line])
    print(s+'\n')
    file.write(s+'\n')
file.close()
    

Rules	support	confidence

area_5 ==> quality_3	54	0.24545454545454545

mach_cat_3 ==> quality_3	79	0.24012158054711247

area_4 ==> quality_3	55	0.23809523809523808

metal_thi_1 ==> quality_1	64	0.2379182156133829

mach_cat_1 ==> quality_2	80	0.23738872403560832

mach_cat_3 ==> quality_1	78	0.23708206686930092

metal_thi_4 ==> quality_3	59	0.23694779116465864

meth_cat_3 ==> quality_3	75	0.2358490566037736

length_5 ==> quality_3	54	0.23275862068965517

meth_cat_1 ==> quality_3	82	0.23229461756373937

area_5 ==> quality_2	51	0.2318181818181818

metal_cat_2 ==> quality_3	78	0.23076923076923078

area_2 ==> quality_1	51	0.22972972972972974

area_2 ==> quality_4	51	0.22972972972972974

meth_cat_2 ==> quality_2	75	0.22796352583586627

metal_thi_3 ==> quality_4	54	0.22406639004149378

metal_thi_3 ==> quality_3	53	0.21991701244813278

length_4 ==> quality_1	51	0.21982758620689655

mach_cat_1 ==> quality_4	74	0.2195845697329377

meth_cat_2 ==> quality_1	72	0.2188449848024316

metal_cat_1 ==> qu